In [1]:
# Configure Jupyter so figures appear in the notebook
%matplotlib inline

# Configure Jupyter to display the assigned value after an assignment
%config InteractiveShell.ast_node_interactivity='last_expr_or_assign'
# import functions from the modsim.py module
from modsim import *
import scipy.signal as sigw

In [2]:
m = UNITS.meter
s = UNITS.second
kg = UNITS.kilogram
A = UNITS.ampere
V = UNITS.volts
Ohm = UNITS.ohm
N = UNITS.newton
U_mu = N / A**2

<Unit('newton / ampere ** 2')>

In [3]:
params = Params(x = 0 * m, 
                d = 1 * m,
                r = 0.1 * m,
                L = 100 * m,
                mass = 1 * kg,
                Vs = 1000 * V,
                R = 0.01 * Ohm,
                mu_0 = 1.257e-6 * U_mu,
                t_end = 100 * s)

,values
x,0 meter
d,1 meter
r,0.1 meter
L,100 meter
mass,1 kilogram
Vs,1000 volt
R,0.01 ohm
mu_0,1.257e-06 newton / ampere ** 2
t_end,100 second


In [4]:
def make_system(params):
    """
    Make a system object.
    
    Parameters:
        params: Params object containing following:
            x: Initial position of armature (in meters)
            d: Separation distance between two rails (in meters)
            r: Radii of the rails (in meters)
            L: Length armature has to travel along rails before launch (in meters)
            mass: Mass of armature (in kilograms)
            Vs: (Initial) Voltage supply magnitude (in volts)
            R: Resistance of railgun circuit (in ohms)
            mu_0: Vacuum permeability constant (in N/A^2)
            t_end: Simulation end time (in seconds)
               
    returns: System object containing
        params values
        c_mag: Calculated magnetic force coefficient (in N/A^2)
        init: Initial state object containing:
            x: Position of armature (initialized to params.x) (in meters)
            v: Velocity of armature (initialized to 0) (in meters)
    """
    
    # make the initial state
    init = State(x=params.x, v=0)
    
    # Determine magnetic force coefficient
    c_mag = params.mu_0 * log((params.d - params.r) / params.r) / (2 * pi)
    
    return System(params, c_mag = c_mag, init=init)

In [5]:
def mag_force(I, system):
    """
    Determines magnetic force on armature due to magnetic loop of railgun. Assumes 1D system.
    
    Parameters:
        I: Current through the circuit at time of function call. (in amperes)
        system: System object containing following system parameters (relevant listed):
            c_mag: Calculated magnetic force coefficient (in N/A^2)
               
    returns: Magnitude of magnetic force
    """
    return system.c_mag * I**2

In [6]:
def slope_func(state, t, system):
    """
    Computes derivatives of the state variables. 
    Invokes function mag_force to determine magnetic force applied on armature.
    
    Parameters:
        state: Current State object at time of function call.
            x: Position of armature (initialized to params.x) (in meters)
            v: Velocity of armature (initialized to 0) (in meters)
        system: System object containing following system parameters (relevant listed):
            Vs: Magnitude of voltage supply to circuit (in volts)
            R: Resistance of the circuit (in ohms)
               
    returns: Magnitude of magnetic force (in newtons)
    """
    x, v = state

    I = system.Vs / system.R
    
    F_mag = mag_force(I, system)
    a = F_mag / system.mass
    
    return v, a

In [7]:
def event_func(state, t, system):
    """
    Stop when the position is equal to the length of rail that can be traveled.
        (stop when the distance left to travel is equal to 0)
    
    Parameters:
        state: Current State object at time of function call.
            x: Position of armature (initialized to params.x) (in meters)
            v: Velocity of armature (initialized to 0) (in meters)
        system: System object containing following system parameters (relevant listed):
            L: Length armature has to travel along rails before launch (in meters)
               
    returns: Distance the armature has left to travel.
    """
    x, v = state
    return x - system.L

In [8]:
def sim_railgun(mass, params):  
    """
    Simulate the railgun model.
    
    Parameters:
        mass: Mass of armature (in kilograms)
        params: Params object containing following:
            x: Initial position of armature (in meters)
            d: Separation distance between two rails (in meters)
            r: Radii of the rails (in meters)
            L: Length armature has to travel along rails before launch (in meters)
            mass: Mass of armature (in kilograms)
            Vs: (Initial) Voltage supply magnitude (in volts)
            R: Resistance of railgun circuit (in ohms)
            mu_0: Vacuum permeability constant (in N/A^2)
            t_end: Simulation end time (in seconds)
               
    returns:
        x: Final position of the armature.
        v: Final velocity of the armature.
    """
    params = Params(params, mass=mass)
    system = make_system(params)
    results, details = run_ode_solver(system, slope_func, events=event_func)
    return results.x, results.v

In [9]:
x, v = sim_railgun(1, params)

In [10]:
x

,values
0.000000,0.000000
0.000100,0.021979
0.001100,2.659410
0.006745,100.000000


In [11]:
v

,values
0.000000,0.000000
0.000100,439.571835
0.001100,4835.290183
0.006745,29650.356989
